#### Pending
1. Processing docx file<br>
2. Implementing peter norvig spell checker<br>
   - In addition to brown dictionary? Or Brown dictionary is to be removed?<br>
   
#### Configuration file
1. The resume file<br>
2. The extra words file<br>
3. Temporary resume file<br>

## Imports

In [ ]:
# Brown dictionary
from nltk.corpus import brown
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# For punctuation marks
import string
# For reading docx file
try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile
# For reading pdf file
import PyPDF2
# For regular expression
import re
from docx import Document

## Reading inbuilt dictionary

In [ ]:
valid_word_set = set(brown.words())

## Read the configuration file that has path of resume, extra dictionary and temporary resume file

In [ ]:
config_file = open(file='./Resume/Resume - Config.txt', mode='r')
for line in config_file:
    if line[:7] == 'resume=':
        resume_path = line[7:]
        # Removing new line character from the end
        resume_path = re.sub(r'[\n\r]+$', '', resume_path)
        # Find the type of resume file
        index_of_last_dot = resume_path.rfind(".")
        resume_type = resume_path[index_of_last_dot+1:]

    if line[:7] == 'exdict=':
        extra_dict_path = line[7:]
        # Removing new line character from the end
        extra_dict_path = re.sub(r'[\n\r]+$', '', extra_dict_path)

    if line[:7] == 'tempre=':
        temp_resume_txt_path = line[7:]
        # Removing new line character from the end
        temp_resume_txt_path = re.sub(r'[\n\r]+$', '', temp_resume_txt_path)

## Read extra words files

In [ ]:
with open(file=extra_dict_path, mode='r') as extra_dict_file:
    extra_dict = set(extra_dict_file.read().split())

# Union of 2 dictionaries
final_good_word_list = extra_dict | valid_word_set

## Read resume file based on the extension

In [ ]:
if resume_type == "txt":
    with open(file=resume_path, mode='r') as resume_file:
        resume_words = resume_file.read()
        
if resume_type == "pdf":
    print('Reading pdf file')
    with open(file=resume_path, mode='rb') as pdf_resume, open(file=temp_resume_txt_path, mode='w') as temp_txt_resume:
        read_pdf = PyPDF2.PdfFileReader(pdf_resume)
        total_pages = read_pdf.getNumPages()
        
        for page_number in range(total_pages):
            page = read_pdf.getPage(page_number)
            page_content = page.extractText()
            temp_txt_resume.write(page_content)
           
    with open(file=temp_resume_txt_path, mode='r') as temp_txt_resume:
        resume_words = temp_txt_resume.read()
        
#if resume_type == "docx" - The logic is pending

## Clean resume

In [ ]:
# Removing numbers
resume_words = re.sub(r'\d+', '', resume_words)
# Removing punctuations
resume_words = re.sub(r'[^a-zA-Z0-9\s]', ' ', resume_words)
# Removing extra new lines and tab
resume_words = re.sub(r"[\n\t]*", "", resume_words)
# Convert all the words that have only first letter as uppercase to all lower case example - "Test" will be converted to "test"
# "TEST" will not be converted
for w in re.findall(r'[A-Z][a-z]+', resume_words):
    resume_words = resume_words.replace(w, w.lower())

## Word tokenization and remove stop words

In [ ]:
tokenized_words = word_tokenize(resume_words)
# Remove Stopwords
stop_words = set(stopwords.words("english"))
tokenized_words = [w for w in tokenized_words if not w in stop_words]

## Find spelling mistake

In [ ]:
wrong_spelling = []

wrong_spelling = [w for w in tokenized_words if not w in final_good_word_list]
# Converting the list to set to eliminate duplicates
wrong_spelling_set = set(wrong_spelling)

print("Total number of spelling errors = ", len(wrong_spelling_set))